In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from torchvision import datasets,transforms

In [2]:
# Creating a transform function object to pre-process the incoming image-data

transforms = transforms.Compose([transforms.ToTensor(),
                                 transforms.Normalize( (0.5,),(0.5,) )])

In [3]:
# Setting the batch_size parameter to chunk our data

batch_size = 64

# Pulling data from torchvision and applying transformations

trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download= True, train= True, transform= transforms)
trainloader = torch.utils.data.DataLoader(dataset= trainset, batch_size= batch_size, shuffle= True)

# Test Dataset

testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download= True, train= False, transform= transforms)
testloader = torch.utils.data.DataLoader(dataset= testset, batch_size= batch_size, shuffle= True)

In [4]:
class FashionNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.hidden1 = nn.Linear(784,256)
    self.hidden2 = nn.Linear(256,128)
    self.output = nn.Linear(128,10)

  def forward(self,x):
    x = F.relu(self.hidden1(x))
    x = F.dropout(x, p= 0.25)
    x = F.relu(self.hidden2(x))
    x = F.dropout(x, p = 0.25)
    output = F.log_softmax(self.output(x))

    return output

In [5]:
model = FashionNetwork()
print(model)

FashionNetwork(
  (hidden1): Linear(in_features=784, out_features=256, bias=True)
  (hidden2): Linear(in_features=256, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=10, bias=True)
)


In [6]:
# Defining the loss function: negative log-likelihood loss

criterion = nn.NLLLoss()

# Defining an optimizer

optimizer = optim.Adam(model.parameters(), lr= 1e-3)

print(optimizer.defaults)

{'lr': 0.001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}


In [7]:
# Writing the training loop

epoch = 10

for _ in range(epoch):
  running_loss = 0
  # Iterating through each image:
  for image,label in trainloader:
    # Initialising the gradients
    optimizer.zero_grad()
    # Reshaping the images
    image = image.view(image.shape[0], -1)
    # Forward Pass of Image
    pred = model(image)
    # Calculating the loss
    loss = criterion(pred,label)
    # Calling .backward function on loss
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  else:
    print(f'Training loss: {running_loss/len(trainloader):.4f}')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


Training loss: 0.5538
Training loss: 0.4208
Training loss: 0.3864
Training loss: 0.3663
Training loss: 0.3507
Training loss: 0.3354
Training loss: 0.3234
Training loss: 0.3166
Training loss: 0.3095
Training loss: 0.3014
